PART 1 - DATA COLLECTION AND CLEANING

First, I read in datasets.

The first dataframe contains country information on demographic and socio-economic.

The second dataframe is compiled form 5 different csv (where each csv represented a different year).  The final dataframe merges these two dataframes, mapping countries in a given year and their various features to a happiness score (for years 2015 to 2017).

In [1]:
import pandas as pd

data_dir = "https://raw.githubusercontent.com/juliandavis7/data/master/"

df_unesco = pd.read_csv(data_dir + "unesco_orig.csv")

df_2015 = pd.read_csv(data_dir + "2015.csv")
df_2016 = pd.read_csv(data_dir + "2016.csv")
df_2017 = pd.read_csv(data_dir + "2017.csv")
df_2018 = pd.read_csv(data_dir + "2018.csv")
df_2019 = pd.read_csv(data_dir + "2019.csv")
df_2015 = df_2015[["Country", "Happiness Score"]]
df_2016 = df_2016[["Country", "Happiness Score"]]
df_2017 = df_2017[["Country", "Happiness.Score"]]
df_2018 = df_2018[["Country or region", "Score"]]
df_2019 = df_2019[["Country or region", "Score"]]

df_2017.rename(columns=
        {"Happiness.Score": "Happiness Score"}, 
        inplace=True)
df_2018.rename(columns=
        {"Country or region": "Country", "Score": "Happiness Score"},
        inplace=True)
df_2019.rename(columns=
        {"Country or region": "Country", "Score": "Happiness Score"},
        inplace=True)

df_2015["Year"] = 2015
df_2016["Year"] = 2016
df_2017["Year"] = 2017
df_2018["Year"] = 2018
df_2019["Year"] = 2019

df_hap = pd.concat([df_2015, df_2016, df_2017, df_2018, df_2019])
df_hap.reset_index(drop=True, inplace=True)

After reading in the data, I have to clean and reorganize the data.  I started by choosing for the indicators that I wanted to investigate further (and the ones that I thought might be correlated with happiness).

In [2]:
df_unesco.head()

,DEMO_IND,Indicator,LOCATION,Country,TIME,Time,Value,Flag Codes,Flags
0,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1970,1970,2.859,NaN,NaN
1,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1971,1971,2.961,NaN,NaN
2,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1972,1972,2.744,NaN,NaN
3,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1973,1973,2.491,NaN,NaN
4,SP_DYN_TFRT_IN,"Fertility rate, total (births per woman)",AUS,Australia,1974,1974,2.397,NaN,NaN


As you can see from the display above, the indicator data for the unesco data was stored in row format.  In this cell, I reorganize the dataframe such that each distinct row (observation) is a given country in a given year.  This requires creating new columns for all the indicators that I desire.

In [3]:
df_final = df_unesco.copy()

df_final = df_unesco[["Country", "Time"]].copy()
df_final.drop_duplicates(inplace=True)
df_final.set_index(["Country", "Time"], inplace=True)

for indicator in df_unesco["Indicator"].unique():
    df_final[indicator] = 0
    df_final[indicator] = df_final[indicator].astype(float)
    
for i, row in df_unesco.iterrows():
    i_country = row["Country"]
    i_year = row["Time"]
    i_feature = row["Indicator"]
    i_val = row["Value"]
    df_final.at[(i_country, i_year), i_feature] = i_val
    
# features I'm interested in looking at
indicators = ["Fertility rate, total (births per woman)",
              "Life expectancy at birth, total (years)",
              "Mortality rate, infant (per 1,000 live births)",
              "Population growth (annual %)",
              "Rural population (% of total population)",
              "GDP growth (annual %)",
              "GDP (current US$)",
              "GDP per capita (current US$)",
              "GDP per capita, PPP (current international $)",
              "GDP, PPP (current international $)",
              "GNI (current LCU)",
              "GNI per capita, Atlas method (current US$)",
              "GNI per capita, PPP (current international $)",
              "Population aged 14 years or younger ",
              "Population aged 15-24 years ",
              "Population aged 25-64 years ",
              "Population aged 65 years or older ",
              "Prevalence of HIV, total (% of population ages 15-49)",
              "Poverty headcount ratio at $1.90 a day (PPP) (% of population)",
              "Total population "]
# comparing GNI to GDP shows the degree to which a nation's GDP 
# represents domestic or international activity

As you can see from the display above, the dataframe is now in the correct format.  Furthermore, I elected to rename the features just to make the dataframe look simpler and cleaner. 

In [44]:
df_unesco = df_final.copy()
df_unesco = df_unesco[indicators].copy()
df_unesco.rename(columns={"Time": "Year",
                         "Fertility rate, total (births per woman)": "fertilityRate",
                         "Life expectancy at birth, total (years)": "lifeExpectancy",
                         "Mortality rate, infant (per 1,000 live births)": "mortalityRate",
                         "Population growth (annual %)": "popGrowth",
                         "Rural population (% of total population)": "ruralPopPct",
                         "GDP growth (annual %)": "gdpGrowthPct",
                         "GDP (current US$)": "gdpUS",
                         "GDP per capita (current US$)": "gdpPerCapitaUS",
                         "GDP per capita, PPP (current international $)": "gdpPerCapita ppp",
                         "GDP, PPP (current international $)": "gdp pppInternational",
                         "GNI (current LCU)": "gni",
                         "GNI per capita, Atlas method (current US$)": "gniPerCapita",
                         "GNI per capita, PPP (current international $)": "gniPerCapita ppp",
                         "Population aged 14 years or younger ": "pop14under",
                         "Population aged 15-24 years ": "pop15to24",
                         "Population aged 25-64 years ": "pop25to64",
                         "Population aged 65 years or older ": "pop65over",
                         "Prevalence of HIV, total (% of population ages 15-49)": "hivPct",
                         "Poverty headcount ratio at $1.90 a day (PPP) (% of population)": "povertyRatio",
                         "Total population ": "totalPop"},
                inplace=True
                )
# drop features with too many missing values
df_unesco.drop(["gdpPerCapita ppp", "gdp pppInternational", 
                "gniPerCapita ppp", "hivPct", "povertyRatio"], 
               axis=1, inplace=True)
df_unesco.head()

fertilityRate  lifeExpectancy  mortalityRate  popGrowth  \
Country   Time                                                            
Australia 1970          2.859        71.01854           17.8    1.97019   
          1971          2.961        71.06829           17.4    3.38029   
          1972          2.744        71.45756           16.8    1.83815   
          1973          2.491        71.84683           16.1    1.52882   
          1974          2.397        72.23610           15.3    2.53122   

                ruralPopPct  gdpGrowthPct         gdpUS  gdpPerCapitaUS  \
Country   Time                                                            
Australia 1970       16.001       7.17218  4.126106e+10      3299.03730   
          1971       15.840       4.00393  4.513831e+10      3489.08660   
          1972       15.680       3.91269  5.195444e+10      3942.81242   
          1973       15.522       2.61382  6.372197e+10      4762.47930   
          1974       15.364       4.10259  8.880914e+10      6471.55416   

                         gni  gniPerCapita  pop14under  pop15to24  pop25to64  \
Country   Time                                                                 
Australia 1970  3.642000e+10        3270.0    3729.907   2241.172   5767.311   
          1971  3.985700e+10        3470.0    3763.468   2288.362   5893.591   
          1972  4.427900e+10        3880.0    3787.515   2321.559   6017.034   
          1973  4.976300e+10        4640.0    3801.483   2344.700   6138.482   
          1974  6.005000e+10        5940.0    3803.345   2365.780   6259.190   

                pop65over   totalPop  
Country   Time                        
Australia 1970   1054.644  12793.034  
          1971   1087.712  13033.133  
          1972   1118.063  13244.171  
          1973   1147.129  13431.794  
          1974   1177.255  13605.570

I noticed that a lot of countries had a lot of slighly different naming conventions so I needed properly reallign all country namings so that the two dataframes, the unesco dataframe and the happiness dataframe, can be merged effectively.  This is an important step to ensure that I retain as much data as possible. The more data I have the better my machine learning models will be able to predict.

- The first cell uses set methods to detect both the countries that are already alligned (using set intersection) and those that are misaligned (using set difference).

- The second cell redefines the country names that were misalligned.

In [45]:
df_unesco.reset_index(inplace=True)
df_unesco.rename(columns = {"Time": "Year"}, inplace=True)
unesco_countries = list(df_unesco["Country"].unique())
hap_countries = list(df_hap["Country"].unique())

def diff(li1, li2): 
    return list(set(li1) - set(li2)) 

unesco_only = diff(unesco_countries, hap_countries)
unesco_only.sort()
hap_only = diff(hap_countries, unesco_countries)
hap_only.sort()

def intersection(li1, li2):
  return list(set(li1) & set(li2))

in_both = intersection(unesco_countries, hap_countries)
print("Countries in both dataframes before allignment:", len(in_both))

Countries in both dataframes before allignment: 141


In [46]:
for i, row in df_unesco.iterrows():
  if df_unesco.at[i, "Country"] == "Bolivia (Plurinational State of)":
    df_unesco.at[i, "Country"] = "Bolivia"
  elif df_unesco.at[i, "Country"] == "Czechia":
    df_unesco.at[i, "Country"] = "Czech Republic"
  elif df_unesco.at[i, "Country"] == "China, Hong Kong Special Administrative Region":
    df_unesco.at[i, "Country"] = "Hong Kong"
  elif df_unesco.at[i, "Country"] == "China, Macao Special Administrative Region":
    df_unesco.at[i, "Country"] = "Hong Kong S.A.R., China"
  elif df_unesco.at[i, "Country"] == "Iran (Islamic Republic of)":
    df_unesco.at[i, "Country"] = "Iran"
  elif df_unesco.at[i, "Country"] == "Russian Federation":
    df_unesco.at[i, "Country"] = "Russia"
  elif df_unesco.at[i, "Country"] == "Republic of Moldova":
    df_unesco.at[i, "Country"] = "Moldova"
  elif df_unesco.at[i, "Country"] == "Palestine":
    df_unesco.at[i, "Country"] = "Palestinian Territories"
  elif df_unesco.at[i, "Country"] == "Republic of Korea":
    df_unesco.at[i, "Country"] = "South Korea"
  elif df_unesco.at[i, "Country"] == "Eswatini":
    df_unesco.at[i, "Country"] = "Swaziland"
  elif df_unesco.at[i, "Country"] == "Syrian Arab Republic":
    df_unesco.at[i, "Country"] = "Syria"
  elif df_unesco.at[i, "Country"] == "United Republic of Tanzania":
    df_unesco.at[i, "Country"] = "Tanzania"
  elif df_unesco.at[i, "Country"] == "United Kingdom of Great Britain and Northern Ireland":
    df_unesco.at[i, "Country"] = "United Kingdom"
  elif df_unesco.at[i, "Country"] == "United States of America":
    df_unesco.at[i, "Country"] = "United States"
  elif df_unesco.at[i, "Country"] == "Venezuela (Bolivarian Republic of)":
    df_unesco.at[i, "Country"] = "Venezuela"
  elif df_unesco.at[i, "Country"] == "Viet Nam":
    df_unesco.at[i, "Country"] = "Vietnam"
  elif df_unesco.at[i, "Country"] == "Democratic Republic of the Congo":
    df_unesco.at[i, "Country"] = "Congo (Kinshasa)"
  elif df_unesco.at[i, "Country"] == "Congo":
    df_unesco.at[i, "Country"] = "Congo (Brazzaville)"
  elif df_unesco.at[i, "Country"] == "Côte d'Ivoire":
    df_unesco.at[i, "Country"] = "Ivory Coast"
  elif df_unesco.at[i, "Country"] == "Lao People's Democratic Republic":
    df_unesco.at[i, "Country"] = "Laos"

  
unesco_countries = list(df_unesco["Country"].unique())
hap_countries = list(df_hap["Country"].unique())

in_both = intersection(unesco_countries, hap_countries)
print("Countries in both dataframes after allignment:", len(in_both))
display(df_hap.head(1))
display(df_unesco.head(1))

Countries in both dataframes after allignment: 161


,Country,Happiness Score,Year
0,Switzerland,7.587,2015


,Country,Year,fertilityRate,lifeExpectancy,mortalityRate,popGrowth,ruralPopPct,gdpGrowthPct,gdpUS,gdpPerCapitaUS,gni,gniPerCapita,pop14under,pop15to24,pop25to64,pop65over,totalPop
0,Australia,1970,2.859,71.01854,17.8,1.97019,16.001,7.17218,4.126106e+10,3299.0373,3.642000e+10,3270.0,3729.907,2241.172,5767.311,1054.644,12793.034


When first trying to apply the machine learning algorithms, I noticed that some of the features I wanted to use had a few missing valus.  In an effort to retain as much data as possible, I did some research and manually filled in these values rather than simply dropping them.  (Note: I only did this for features that had < 10 missing values)

In [47]:
df_unesco.set_index(["Country", "Year"], inplace=True)

df_unesco.at[("Syria", 2015), "gdpGrowthPct"] = -6.1
df_unesco.at[("Syria", 2015), "gdpUS"] = 19090000
df_unesco.at[("Syria", 2015), "gdpPerCapitaUS"] = 890
df_unesco.at[("Syria", 2015), "gniPerCapita"] = 681

df_unesco.at[("Syria", 2016), "gdpGrowthPct"] = -4.0
df_unesco.at[("Syria", 2016), "gdpUS"] = 12377000
df_unesco.at[("Syria", 2016), "gdpPerCapitaUS"] = 709
df_unesco.at[("Syria", 2016), "gniPerCapita"] = 377

df_unesco.at[("Syria", 2017), "gdpGrowthPct"] = 1.9
df_unesco.at[("Syria", 2017), "gdpUS"] = 15183000
df_unesco.at[("Syria", 2017), "gdpPerCapitaUS"] = 890
df_unesco.at[("Syria", 2017), "gniPerCapita"] = 704

df_unesco.at[("Venezuela", 2015), "gdpGrowthPct"] = -6.2
df_unesco.at[("Venezuela", 2015), "gdpUS"] = 323595000000
df_unesco.at[("Venezuela", 2015), "gdpPerCapitaUS"] = 10568.1
df_unesco.at[("Venezuela", 2015), "gniPerCapita"] = 11047

df_unesco.at[("Venezuela", 2016), "gdpGrowthPct"] = -17.04
df_unesco.at[("Venezuela", 2016), "gdpUS"] = 279249000000
df_unesco.at[("Venezuela", 2016), "gdpPerCapitaUS"] = 9092.02
df_unesco.at[("Venezuela", 2016), "gniPerCapita"] = 9420

df_unesco.at[("Venezuela", 2017), "gdpGrowthPct"] = -15.67
df_unesco.at[("Venezuela", 2017), "gdpUS"] = 143841000000
df_unesco.at[("Venezuela", 2017), "gdpPerCapitaUS"] = 4755.03
df_unesco.at[("Venezuela", 2017), "gniPerCapita"] = 8216

df_unesco.at[("Somalia", 2016), "gdpGrowthPct"] = 2.89
df_unesco.at[("Somalia", 2016), "gniPerCapita"] = 101

df_unesco.at[("Somalia", 2017), "gdpGrowthPct"] = 1.39
df_unesco.at[("Somalia", 2017), "gniPerCapita"] = 102

df_unesco.at[("Lithuania", 2015), "gni"] = 43900000000
df_unesco.at[("Lithuania", 2015), "gniPerCapita"] = 15110

df_unesco.at[("Lithuania", 2016), "gni"] = 42460000000
df_unesco.at[("Lithuania", 2016), "gniPerCapita"] = 14800

df_unesco.at[("Lithuania", 2017), "gni"] = 42930000000
df_unesco.at[("Lithuania", 2017), "gniPerCapita"] = 15180

df_unesco.at[("Yemen", 2017), "gniPerCapita"] = 1060

Finally, I merge the two data frames on country and year.  My dataframe - containing the data and their corresponding labels (happiness score) - can now easily be applied to machine learning regression algorithms.

Additionally, all the columns were already associated with the correct data type so no further manipulation was needed there.

In [48]:
df_combined = df_unesco.merge(df_hap, on=["Country", "Year"])
df_combined.rename(columns={"Happiness Score": "happinessScore"}, inplace=True)
df_combined = df_combined[(df_combined["Year"] == 2015) |
                          (df_combined["Year"] == 2016) |
                          (df_combined["Year"] == 2017)]
# 2018 and 2019 are missing information, cannot be used

df_combined.set_index(["Country", "Year"], inplace=True)
df_combined.dtypes

fertilityRate     float64
lifeExpectancy    float64
mortalityRate     float64
popGrowth         float64
ruralPopPct       float64
gdpGrowthPct      float64
gdpUS             float64
gdpPerCapitaUS    float64
gni               float64
gniPerCapita      float64
pop14under        float64
pop15to24         float64
pop25to64         float64
pop65over         float64
totalPop          float64
happinessScore    float64
dtype: object

Finally, I downloaded the fully cleaned dataframe to a .csv in my data directory.

In [50]:
df_unesco.to_csv("unesco.csv")
df_combined.to_csv("unesco_train.csv")

fertilityRate  lifeExpectancy  mortalityRate  \
Country                 Year                                                 
Australia               2015          1.814        82.40000            3.2   
                        2016          1.752        82.44878            3.1   
                        2017          1.765        82.49756            3.0   
Austria                 2015          1.490        81.19024            3.0   
                        2016          1.530        81.64146            3.0   
...                                     ...             ...            ...   
Somalia                 2016          6.267        56.29300           82.1   
                        2017          6.171        56.71400           79.7   
Palestinian Territories 2015          4.075        73.29900           18.6   
                        2016          4.008        73.47300           18.3   
                        2017          3.942        73.64600           17.9   

                              popGrowth  ruralPopPct  gdpGrowthPct  \
Country                 Year                                         
Australia               2015    1.43922       14.299       2.35114   
                        2016    1.56194       14.200       2.82731   
                        2017    1.68452       14.096       1.95758   
Austria                 2015    1.12099       42.285       1.14298   
                        2016    1.08140       42.095       2.03957   
...                                 ...          ...           ...   
Somalia                 2016    2.90437       56.184       2.89000   
                        2017    2.92188       55.609       1.39000   
Palestinian Territories 2015    2.92469       24.632       3.42873   
                        2016    2.88469       24.372       4.70844   
                        2017    2.88470       24.106       3.14024   

                                     gdpUS  gdpPerCapitaUS           gni  \
Country                 Year                                               
Australia               2015  1.349034e+12     56644.03396  1.587954e+12   
                        2016  1.208039e+12     49937.73139  1.622282e+12   
                        2017  1.323421e+12     53793.53726  1.707621e+12   
Austria                 2015  3.818057e+11     44176.67174  3.408691e+11   
                        2016  3.940528e+11     45103.32981  3.558314e+11   
...                                    ...             ...           ...   
Somalia                 2016  6.761999e+09       472.27269  1.552591e+14   
                        2017  7.052000e+09       478.34417  1.621771e+14   
Palestinian Territories 2015  1.267300e+10      2865.80511  1.438530e+10   
                        2016  1.342570e+10      2949.68809  1.532190e+10   
                        2017  1.449810e+10      3094.72575  1.649000e+10   

                              gniPerCapita  pop14under  pop15to24  pop25to64  \
Country                 Year                                                   
Australia               2015       60440.0    4520.213   3172.395  12684.814   
                        2016       54180.0    4598.344   3155.240  12835.482   
                        2017       51360.0    4687.067   3135.675  12974.858   
Austria                 2015       47490.0    1225.425   1008.434   4809.700   
                        2016       46130.0    1239.550   1001.790   4851.921   
...                                    ...         ...        ...        ...   
Somalia                 2016         101.0    6659.213   2927.672   4197.453   
                        2017         102.0    6820.876   3033.500   4318.474   
Palestinian Territories 2015        3260.0    1807.277    985.696   1599.578   
                        2016        3320.0    1835.116    995.881   1663.021   
                        2017        3340.0    1863.891   1002.570   1733.883   

                              pop65over   totalPop  happinessScore  
Count